# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via a cargo plane to there, and represent them on a map, with a color varying by a cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

Let's build this!

In [1]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 30.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
!pip install markdownify

In [21]:
!pip install requests

In [12]:
!pip install duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.1 MB/s eta 0:00:00


In [22]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [23]:
import os
from PIL import Image
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, VisitWebpageTool


model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [24]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [9]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [25]:
agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [26]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search("Batman filming locations")                                                        
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of Gotham
City are located in Liverpool, although Chicago was ...

[Where The Batman Filmed (All Gotham Real-Life 
Locations)](https://screenrant.com/where-was-the-batman-movie-filmed/)
Why The Batman's Reviews Are So Positive The Batman did its best to film in real-life locations, although prominent
movie locations like the Batcave and Wayne Tower were built as sets on studio lots. When the movie was filming on 
location, the cast and crew mostly stayed in the UK to film the biggest Gotham locations.

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
Travel guide to film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
For The Batman, Matt Reeves mainly filmed in the UK and Chicago to capture the cold, gothic, and older feel of 
Gotham. While The Batman did try to film in real locations, iconic spots like the Batcave and Wayne Tower were 
built on studio sets. When filming on location, the cast and crew spent most of their time in the UK for the main 
Gotham scenes.

[Batman Movie Locations You Can Visit In Person! - 
Travel](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Gotham city is fictional, but you can visit these batman movie locations to relive some of the iconic scenes.

[Where Was The Batman Filmed? All the Iconic Locations 
Explored](https://otakukart.com/where-was-the-batman-filmed-all-the-iconic-locations-explored/)
The Batman combines real-world locations and digital effects to create a compelling and atmospheric rendition of 
Gotham City. Whether filming in the UK or Chicago, the filmmakers utilized a diverse array of settings to bring the
dark and gritty world of Gotham to life.

[The Batman Filming Locations - filmiloc.com](https://www.filmiloc.com/the-batman-filming-locations/)
Matt Reeves' 'The Batman' was filmed in many places around the world to show a new version of Gotham City. This 
movie brings Robert Pattinson as Batman and shows his world. It also includes characters like Penguin, Catwoman, 
and others from DC Comics. Gotham City itself becomes a very important character in the movie. Filming on real 
locations helped create this new version of Gotham.

[The Batman Filming Locations (2022)](https://a2zfilminglocation.com/the-batman-filming-locations-2022/)
The Batman Filming Locations: The Batman is a 2022 American superhero film that follows a sadistic serial killer 
who starts to murder all key political figures in Gotham along with it Batman is forced to investigate the city's 
hidden corruption and also his family's involvement. The Batman film stars Robert Pattinson as Batman along with 
Zoë Kravitz, Paul Dano, Jeffrey Wright, John ...

[Where Was The Batman Filmed? 2022 Movie Filming 
Locations](https://thecinemaholic.com/where-was-the-batman-filmed/)
The Batman Filming Locations 'The Batman' was filmed in various parts of England and Scotland, specifically in 
Liverpool, London, Watford, Bedford, Glasgow, and Shotts.

[Where Was The Batman Filmed? | iFILMthings](https://ifilmthings.com/where-was-the-batman-filmed/)
Where Was The Batman Filmed? The Batman movie was filmed in various locations around the world, including London 
and Liverpool in England, Chicago in the Uni

[Step 1: Duration 3.98 seconds| Input tokens: 2,274 | Output tokens: 167]

...

[Step 4: Duration 13.90 seconds| Input tokens: 52,354 | Output tokens: 1,728]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
422 Client Error: Unprocessable Entity for url: https://router.huggingface.co/together/v1/chat/completions (Request
ID: nuZLyov-4yUbBN-941cebfd9fea3b65)

{'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 38787 `inputs` 
tokens and 2048 `max_new_tokens`', 'type': 'invalid_request_error', 'param': None, 'code': None}
{
  "id": "nuZLyov-4yUbBN-941cebfd9fea3b65",
  "error": {
    "message": "Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 38787 `inputs` 
tokens and 2048 `max_new_tokens`",
    "type": "invalid_request_error",
    "param": null,
    "code": null
  }
}

[Step 5: Duration 1.07 seconds| Input tokens: 76,746 | Output tokens: 2,195]

AgentGenerationError: Error in generating model output:
422 Client Error: Unprocessable Entity for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: nuZLyov-4yUbBN-941cebfd9fea3b65)

{'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 38787 `inputs` tokens and 2048 `max_new_tokens`', 'type': 'invalid_request_error', 'param': None, 'code': None}
{
  "id": "nuZLyov-4yUbBN-941cebfd9fea3b65",
  "error": {
    "message": "Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 38787 `inputs` tokens and 2048 `max_new_tokens`",
    "type": "invalid_request_error",
    "param": null,
    "code": null
  }
}


In [ ]:
result

,Location,Travel Time to Gotham (hours)
0,"Necropolis Cemetery, Glasgow, Scotland, UK",8.60
1,"St. George's Hall, Liverpool, England, UK",8.81
2,"Two Temple Place, London, England, UK",9.17
3,"Wollaton Hall, Nottingham, England, UK",9.00
4,"Knebworth House, Knebworth, Hertfordshire, Eng...",9.15
5,"Acton Lane Power Station, Acton Lane, Acton, E...",9.16
6,"Queensboro Bridge, New York City, USA",1.01
7,"Wall Street, New York City, USA",1.00
8,"Mehrangarh Fort, Jodhpur, Rajasthan, India",18.34
9,"Turda Gorge, Turda, Romania",11.89


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [27]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Gotham coordinates: 40.7128° N, 74.0060° W
- We need to find Batman filming locations worldwide.
- We need to calculate the travel time for a cargo plane to Gotham for each filming location.
- We need to find supercar factories with the same cargo plane transfer time to Gotham.

### 1.2. Facts to look up
- List of Batman filming locations worldwide.
  - Source: Web search for "Batman filming locations" and verify from reliable sources like IMDb, official movie 
websites, or reputable news articles.
- Coordinates of each Batman filming location.
  - Source: Geocoding services or verified maps and location databases.
- List of supercar factories worldwide.
  - Source: Web search for "supercar factories" and verify from reliable sources like automotive industry websites 
or reputable news articles.
- Coordinates of each supercar factory.
  - Source: Geocoding services or verified maps and location databases.
- Cruising speed of a typical cargo plane.
  - Source: Aviation industry standards or reliable aviation websites.

### 1.3. Facts to derive
- Travel time for a cargo plane from each Batman filming location to Gotham.
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of each filming location and 
Gotham.
- Travel time for a cargo plane from each supercar factory to Gotham.
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of each supercar factory and 
Gotham.

## 2. Plan
1. Perform a web search for "Batman filming locations" and gather a list of locations.
2. For each Batman filming location, perform a web search to find its coordinates.
3. Perform a web search for "supercar factories" and gather a list of factories.
4. For each supercar factory, perform a web search to find its coordinates.
5. Perform a web search to find the cruising speed of a typical cargo plane.
6. Use the `calculate_cargo_travel_time` function to calculate the travel time from each Batman filming location to
Gotham.
7. Use the `calculate_cargo_travel_time` function to calculate the travel time from each supercar factory to 
Gotham.
8. Create a pandas dataframe with Batman filming locations and their travel times to Gotham.
9. Create a pandas dataframe with supercar factories and their travel times to Gotham.
10. Finalize the dataframes and prepare them for the final answer.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Gather Batman filming locations                                                                        
  batman_locations_query = "Batman filming locations"                                                              
  batman_locations = web_search(batman_locations_query)                                                            
  print("Batman filming locations:", batman_locations)                                                             
                                                                                                                   
  # Step 2: Gather supercar factories                                                                              
  supercar_factories_query = "supercar factories"                                                                  
  supercar_factories = web_search(supercar_factories_query)                                                        
  print("Supercar factories:", supercar_factories)                                                                 
                                                                                                                   
  # Step 3: Gather cruising speed of a typical cargo plane                                                         
  cruising_speed_query = "cruising speed of a typical cargo plane"                                                 
  cruising_speed = web_search(cruising_speed_query)                                                                
  print("Cruising speed of a typical cargo plane:", cruising_speed)                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Batman filming locations: ## Search Results

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of Gotham
City are located in Liverpool, although Chicago was ...

[Where The Batman Filmed (All Gotham Real-Life 
Locations)](https://screenrant.com/where-was-the-batman-movie-filmed/)
Why The Batman's Reviews Are So Positive The Batman did its best to film in real-life locations, although prominent
movie locations like the Batcave and Wayne Tower were built as sets on studio lots. When the movie was filming on 
location, the cast and crew mostly stayed in the UK to film the biggest Gotham locations.

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
Travel guide to film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

[Batman Movie Locations You Can Visit In Person! - 
Travel](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Gotham city is fictional, but you can visit these batman movie locations to relive some of the iconic scenes.

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
For The Batman, Matt Reeves mainly filmed in the UK and Chicago to capture the cold, gothic, and older feel of 
Gotham. While The Batman did try to film in real locations, iconic spots like the Batcave and Wayne Tower were 
built on studio sets. When filming on location, the cast and crew spent most of their time in the UK for the main 
Gotham scenes.

[Where Was The Batman Filmed? All the Iconic Locations 
Explored](https://otakukart.com/where-was-the-batman-filmed-all-the-iconic-locations-explored/)
The Batman combines real-world locations and digital effects to create a compelling and atmospheric rendition of 
Gotham City. Whether filming in the UK or Chicago, the filmmakers utilized a diverse array of settings to bring the
dark and gritty world of Gotham to life.

[The Batman Filming Locations - filmiloc.com](https://www.filmiloc.com/the-batman-filming-locations/)
Matt Reeves' 'The Batman' was filmed in many places around the world to show a new version of Gotham City. This 
movie brings Robert Pattinson as Batman and shows his world. It also includes characters like Penguin, Catwoman, 
and others from DC Comics. Gotham City itself becomes a very important character in the movie. Filming on real 
locations helped create this new version of Gotham.

[The Batman Filming Locations (2022)](https://a2zfilminglocation.com/the-batman-filming-locations-2022/)
The Batman Filming Locations: The Batman is a 2022 American superhero film that follows a sadistic serial killer 
who starts to murder all key political figures in Gotham along with it Batman is forced to investigate the city's 
hidden corruption and also his family's involvement. The Batman film stars Robert Pattinson as Batman along with 
Zoë Kravitz, Paul Dano, Jeffrey Wright, John ...

[Where Was The Batman Filmed? 2022 Movie Filming 
Locations](https://thecinemaholic.com/where-was-the-batman-filmed/)
The Batman Filming Locations 'The Batman' was filmed in various parts of England and Scotland, specifically in 
Liverpool, London, Watford, Bedford, Glasgow, and Shotts.

[Where Was The Batman Filmed? | iFILMthings](https://ifilmthings.com/where-was-the-batman-filmed/)
Where Was The Batman Filmed? The Batman movie was filmed in various locations around the world, including London 
and Liverpool in E

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Gotham coordinates                                                                                             
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Batman filming locations and their URLs                                                                        
  batman_locations = [                                                                                             
      ("https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html", "Liverpool, UK"),                
      ("https://screenrant.com/where-was-the-batman-movie-filmed/", "London, UK"),                                 
      ("https://www.movie-locations.com/movies/b/The-Batman-2022.php", "Glasgow, UK"),                             
      ("https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/",           
  "Chicago, USA")                                                                                                  
  ]                                                                                                                
                                                                                                                   
  # Supercar factories and their URLs                                                                              
  supercar_factories = [                                                                                           
      ("https://supercarblondie.com/inside-the-mclaren-supercar-factory-in-woking-england/", "McLaren, Woking,     
  UK"),                                                                                                            
      ("https://www.msn.com/en-us/video/peopleandplaces/step-inside-the-lamborghini-factory-italy-s-supercar-asse  
  mbly-line/vi-AA1EZG4I", "Lamborghini, Sant'Agata Bolognese, Italy"),                                             
      ("https://jalopnik.com/inside-the-only-supercar-factory-in-america-1830088702/", "Noble, West Molesey,       
  UK"),                                                                                                            
      ("https://www.supercarworld.com/cgi-bin/listall.cgi?make", "Pagani, San Cesario sul Panaro, Italy"),         
      ("https://www.cnet.com/roadshow/pictures/inside-mclaren/", "Ferrari, Maranello, Italy")                      
  ]                                                                                                                
                                                                                                                   
  # Function to extract coordinates from a webpage                                                                 
  def extract_coordinates(page_content):                                                                           
      # Example regex to extract coordinates (this may need to be adjusted for each page)                          
      coordinates = re.search(r"(\d+\.\d+)\s*°\s*[NS],\s*(\d+\.\d+)\s*°\s*[WE]", page_content)                     
      if coordinates:                                                                                              
          lat, lon = map(float, coordinates.groups())                                                              
          return lat, lon                                                                                          
      return None                                        

Out - Final answer:                                                                           URL  \
Batman Filming Locations 0  https://www.atlasofwonders.com/2022/04/where-w...   
                         1  https://screenrant.com/where-was-the-batman-mo...   
                         2  https://www.movie-locations.com/movies/b/The-B...   
                         3  https://www.travelandleisureasia.com/global/de...   
Supercar Factories       0  https://supercarblondie.com/inside-the-mclaren...   
                         2  https://jalopnik.com/inside-the-only-supercar-...   

                                           Location  \
Batman Filming Locations 0            Liverpool, UK   
                         1               London, UK   
                         2              Glasgow, UK   
                         3             Chicago, USA   
Supercar Factories       0      McLaren, Woking, UK   
                         2  Noble, West Molesey, UK   

                            Travel Time to Gotham (hours)  
Batman Filming Locations 0                          10.15  
                         1                          10.57  
                         2                           9.91  
                         3                           2.97  
Supercar Factories       0                          10.53  
                         2                          10.56  

[Step 7: Duration 29.63 seconds| Input tokens: 103,156 | Output tokens: 9,440]

                                                                          URL  \
Batman Filming Locations 0  https://www.atlasofwonders.com/2022/04/where-w...   
                         1  https://screenrant.com/where-was-the-batman-mo...   
                         2  https://www.movie-locations.com/movies/b/The-B...   
                         3  https://www.travelandleisureasia.com/global/de...   
Supercar Factories       0  https://supercarblondie.com/inside-the-mclaren...   
                         2  https://jalopnik.com/inside-the-only-supercar-...   

                                           Location  \
Batman Filming Locations 0            Liverpool, UK   
                         1               London, UK   
                         2              Glasgow, UK   
                         3             Chicago, USA   
Supercar Factories       0      McLaren, Woking, UK   
                         2  Noble, West Molesey, UK   

                            Travel Time to Got

In [ ]:
detailed_report

,Location,Travel Time (hours)
0,"Bridge of Sighs, Glasgow Necropolis, Glasgow, ...",8.6
1,"Wishart Street, Glasgow, Scotland, UK",8.6


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [28]:
model = InferenceClientModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [29]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [30]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=InferenceClientModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [31]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a duckduckgo web search  │ query (`string`): The search      │
            │                             │ based on your query (think a      │ query to perform.                 │
            │                             │ Google search) then returns the   │                                   │
            │                             │ top search results.               │                                   │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.                         │ destination_coords (`array`):     │
            │                             │               

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ InferenceClientModel - deepseek-ai/DeepSeek-R1 ────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
<think>
Okay, let's tackle this task step by step. First, I need to understand what's required. The user wants all Batman 
filming locations worldwide, calculate the cargo plane travel time from Gotham (40.7128° N, 74.0060° W) to each of 
those locations. Then, find supercar factories that have the same travel time as those Batman locations, ensuring 
at least six points in total. Finally, create a map visualizing these points with color-coded travel times and save
it as saved_map.png.

Starting with the facts survey. The given facts are the coordinates of Gotham, which is actually New York City's 
coordinates. The example code for plotting uses plotly.express, so that's a hint on how to structure the map.

For the facts to look up, first, I need a list of all Batman filming locations. Since Batman movies have been 
filmed in various places over the years, I'll need to search for each movie's filming locations. For example, "The 
Dark Knight" had scenes in Chicago, London, etc. The web_agent can help here by querying reliable sources like IMDb
or official movie databases.

Next, for each filming location found, I need their exact coordinates. Some might be well-known, like Chicago's 
coordinates, but others might require more precise lookup. Using the web_agent again to get latitude and longitude 
for each city or specific site mentioned.

Then, supercar factories. I need to find manufacturers of supercars (like Ferrari in Italy, Lamborghini in 
Sant'Agata Bolognese, McLaren in the UK, etc.). For each factory, get their coordinates. But the key is to find 
factories whose travel time from Gotham matches any of the Batman locations' times. So after calculating the times 
for Batman locations, I need to compute the required time and then find factories with the same or similar times.

Facts to derive include the travel times for each Batman location using the calculate_cargo_travel_time function. 
Once those times are known, find supercar factories that have the same travel time. If there aren't enough 
factories with exact times, maybe allow a small margin of error. Then combine all these points (Batman locations 
and matching factories) into a dataset for plotting.

The plan would be:

1. Use the web_agent to gather all Batman filming locations. This might involve searching each major Batman movie 
and compiling their filming sites.
2. For each location found, get their coordinates via web_agent. For example, if a scene was filmed in Chicago, get
Chicago's lat/long.
3. Calculate the travel time from Gotham to each Batman location using the provided function. The cruising speed 
defaults to 750 km/h.
4. Now, find supercar factories. Use web_agent to list known supercar manufacturers and their factory locations. 
Get their coordinates.
5. For each factory, calculate the travel time from Gotham. Compare these times to the Batman locations' times. 
Select factories that match any of the Batman times. If needed, adjust to get at least six points in total.
6. Compile all Batman locations and matching factories into a dataset with their coordinates, names, and travel 
times.
7. Use plotly to create a scatter map. Assign colors based on travel time. Ensure the example code is adapted to 
use the correct columns (lat, lon, color, etc.) from the dataset.
8. Save the map as saved_map.png and return it using final_answer.

Potential issues: Ensuring enough supercar factories match the travel times. If not, maybe relax the matching 
criteria (e.g., within 30 minutes). Also, verifying the accuracy of coordinates for filming locations, as some 
might be specific landmarks rather than cities. The web_agent needs to find precise data here.

Another point: The example code uses a dataframe with 'centroid_lat', 'centroid_lon', 'name', and 'pe

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<think>                                                                                                            
Okay, let's tackle this step by step. First, I need to get all the Batman filming locations. I'll use the web_agent
tool to search for a list of all Batman movie filming locations with their cities. Once I have that list, I'll need
to get the coordinates for each location. For example, if one of the locations is Chicago, I'll ask the web_agent  
for its exact coordinates.                                                                                         
                                                                                                                   
Next, I'll calculate the travel time from Gotham (40.7128° N, 74.0060° W) to each of these locations using the     
calculate_cargo_travel_time function. The cruising speed is 750 km/h by default, so I don't need to specify that   
unless there's a different speed mentioned, which there isn't here.                                                
                                                                                                                   
Then, I need to find supercar factories. I'll use the web_agent again to get a list of supercar manufacturers and  
their factory locations. Once I have those, I'll get their coordinates too. After that, I'll calculate the travel  
time from Gotham to each factory using the same function.                                                          
                                                                                                                   
Now, I need to match the factories' travel times to the Batman locations' times. The goal is to have at least six  
points in total. If there aren't[38;2;230;237;243;4

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Get Batman filming locations                                                                                   
  batman_locations = web_agent("Provide a comprehensive list of all Batman movie filming locations (1989-2023)     
  with city names and countries. Include exact filming site names where possible.")                                
  print("Batman locations raw data:", batman_locations)                                                            
                                                                                                                   
  # Get coordinates for each location (example for one location - will need to loop through all)                   
  # This is a template - actual implementation would process all locations                                         
  gotham_coords = (40.7128, -74.0060)  # Our origin point                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<think>                                                                                                            
                                                                                                                   
                                                                                                                   
Thought: Since plotly.io is restricted, I'll use the Figure object from plotly directly and save using the         
write_image method.                                                                                                
                                                                                                                   
Code:                                                                                                              
```python                                                                                                          
import plotly                                                                                                      
from plotly.graph_objs import Figure, Scattermapbox, Layout                                                        
                                                                                                                   
# Combined data                                                                                                    
combined = [                                                                                                       
    {"name": "Chicago Board of Trade", "lat": 41.8781, "lon": -87.6298, "time": 1.5},                              
    {"name": "Chicago Riverwalk", "lat": 41.8875, "lon": -87.6228, "time": 1.5},                                   
    {"name": "Elstree Studios UK", "lat": 51.6586, "lon": -0.2793, "time": 6.8},                                   
    {"name": "Millennium Park Chicago", "lat": 41.8825, "lon": -87.6225, "time": 1.5},                             
    {"name": "McLaren Woking", "lat": 51.3160, "lon": -0.5632, "time": 6.8}                                        
]                                                                                                                  
                                                                                                                   
# Create figure                                                                                                    
fig = Figure(data=[                                                                                                
    Scattermapbox(                                                                                                 
        lat=[loc["lat"] for loc in combined],                                                                      
        lon=[loc["lon"] for loc in combined],                                                                      
        mode='markers',                                                                                            
        marker=dict(                                                                                               
            size=14,                                                                                               
            color=[loc["time"] for loc in combined],                                                               
            colorscale='Plasma',                                                                                   
            showscale=True                                                                                         
        ),                                                                                                         
    [38;2;230;237;243;48;2

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import plotly                                                                                                    
  from plotly.graph_objs import Figure, Scattermapbox, Layout                                                      
                                                                                                                   
  # Combined data                                                                                                  
  combined = [                                                                                                     
      {"name": "Chicago Board of Trade", "lat": 41.8781, "lon": -87.6298, "time": 1.5},                            
      {"name": "Chicago Riverwalk", "lat": 41.8875, "lon": -87.6228, "time": 1.5},                                 
      {"name": "Elstree Studios UK", "lat": 51.6586, "lon": -0.2793, "time": 6.8},                                 
      {"name": "Millennium Park Chicago", "lat": 41.8825, "lon": -87.6225, "time": 1.5},                           
      {"name": "McLaren Woking", "lat": 51.3160, "lon": -0.5632, "time": 6.8}                                      
  ]                                                                                                                
                                                                                                                   
  # Create figure                                                                                                  
  fig = Figure(data=[                                                                                              
      Scattermapbox(                                                                                               
          lat=[loc["lat"] for loc in combined],                                                                    
          lon=[loc["lon"] for loc in combined],                                                                    
          mode='markers',                                                                                          
          marker=dict(                                                                                             
              size=14,                                                                                             
              color=[loc["time"] for loc in combined],                                                             
              colorscale='Plasma',                                                                                 
              showscale=True                                                                                       
          ),                                                                                                       
          text=[loc["name"] for loc in combined],                                                                  
          hoverinfo='[38;2;230;2

Code execution failed at line 'from plotly.graph_objs import Figure, Scattermapbox, Layout' due to: 
InterpreterError: Import from plotly.graph_objs is not allowed. Authorized imports are: ['collections', 'queue', 
'numpy', 'time', 'geopandas', 'datetime', 'statistics', 'plotly', 'math', 'pandas', 'json', 're', 'unicodedata', 
'stat', 'shapely', 'random', 'itertools'\]

[Step 9: Duration 6.15 seconds| Input tokens: 88,372 | Output tokens: 10,180]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)